In [68]:
# 原始資料來源
# http://147.8.142.179/datazip/
import os, sys
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
import jieba.posseg as pseg
# jieba 設定斷詞
jieba.set_dictionary('jieba/dict.txt.big')
# jieba 設定停止詞
jieba.analyse.set_stop_words("jieba/stop_words.txt")
from pyspark import SparkContext
from pyspark.mllib.feature import Word2Vec


In [84]:
# 啟動 jieba分散式
jieba.enable_parallel(4)

Building prefix dict from /Users/SkyTim/spark/jieba/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /Users/SkyTim/spark/jieba/dict.txt.big ...
Dumping model to file cache /var/folders/w3/kbpct83955524gxmzln4r4vr0000gn/T/jieba.u5005e2b038b70f7b91dba6e0bb094401.cache
DEBUG:jieba:Dumping model to file cache /var/folders/w3/kbpct83955524gxmzln4r4vr0000gn/T/jieba.u5005e2b038b70f7b91dba6e0bb094401.cache
Loading model cost 9.541 seconds.
DEBUG:jieba:Loading model cost 9.541 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [105]:
# 斷詞並且將字詞為名詞的字詞找出來
def intest(T):
    psegCut=pseg.cut(T)
    words=[ ]
    for word ,flag in psegCut:
        if(flag=="n"):
            words.append(word)
    return "/".join(words)

In [106]:
# 去除掉空白
def nottest(T):
    if(T!=''):
        return T.split("/")

In [107]:
# 去除None
def noneTest(T):
    if(T!=None):
        return T

In [116]:
# 搜集資料 過濾掉字詞小於 8 欄位在第6
documents = sc.textFile("data/week1.csv").map(lambda line:line.split(",")).filter(lambda x:len(x)>8 ).map(lambda t:t[6]).map(lambda words:intest(words)).map(lambda row: nottest(row)).filter(lambda ct:not ct==None  )

In [133]:
# 格式為
for word in documents.take(10):
    print word

[u'\u535a']
[u'\u5168\u90fd']
[u'\u535a']
[u'\u4eba']
[u'\u5927\u5bb6']
[u'\u8863\u670d']
[u'\u9738\u6c14', u'\u9738\u6c14']
[u'\u6837\u5b50', u'\u4efb\u52a1', u'\u6b4c\u624b', u'\u8bb2\u8bdd', u'\u5706\u573a', u'\u8ff7\u5bab', u'\u78c1\u77f3', u'\u773c\u795e', u'\u4f20\u60c5']
[u'\u5fc3', u'\u5fc3', u'\u5fc3']
[u'\u82b1\u5fc3', u'\u82b1\u5fc3', u'\u535a']


In [123]:
# train model
word2vec = Word2Vec()
model = word2vec.fit(documents)


In [127]:
# 找出與股市相關的20個字詞
synonyms = model.findSynonyms('股市', 20)

In [135]:
# 與'股市'相關的字詞列印出來
for word, cosine_distance in synonyms:
    print word+", heavy is "+str(cosine_distance)


A股, heavy is 1.98845765004
全球股市, heavy is 1.96794519601
股票, heavy is 1.93961700215
交易日, heavy is 1.91831716695
投资者, heavy is 1.90264466867
美国股市, heavy is 1.80945048204
晴雨表, heavy is 1.78571162065
高点, heavy is 1.7813645395
股民, heavy is 1.77996181412
走势, heavy is 1.77038924681
低点, heavy is 1.76461427364
证券, heavy is 1.76020417001
圈钱, heavy is 1.7449535243
股指, heavy is 1.74150508436
午盘, heavy is 1.74110680524
涨, heavy is 1.73896247939
新股, heavy is 1.72332987906
金融股, heavy is 1.70558093187
散户, heavy is 1.7014143669
跌势, heavy is 1.7002939989


In [137]:
# 將model存起來
model.save(sc,"model")